In [13]:
import pandas as pd
df = pd.read_csv("history.csv", index_col=0)
df.rename(columns={"into": "open", "inth": "high", "intc": "close", "intl": "low"}, inplace=True)
df = df[["time", "open", "high", "low", "close"]]
df = df.iloc[::-1]
df

,time,open,high,low,close
114,26-03-2024 09:15:00,46638.75,46782.15,46638.75,46738.05
113,26-03-2024 09:30:00,46740.50,46778.40,46684.10,46713.45
112,26-03-2024 09:45:00,46714.10,46738.15,46667.95,46689.30
111,26-03-2024 10:00:00,46688.25,46716.30,46594.05,46638.15
110,26-03-2024 10:15:00,46634.55,46675.50,46605.50,46661.80
...,...,...,...,...,...
4,02-04-2024 11:45:00,47607.05,47627.65,47586.80,47603.70
3,02-04-2024 12:00:00,47603.85,47628.95,47582.30,47598.85
2,02-04-2024 12:15:00,47597.65,47624.10,47579.25,47589.10
1,02-04-2024 12:30:00,47590.10,47663.10,47590.10,47651.50


In [8]:
from streaming_indicators import CPR
cpr_obj = CPR()
for i, row in df.iterrows():
    row_as_dict = row.to_dict()
    row_as_dict
    print(cpr_obj.compute(row_as_dict))

(47625.05, 47633.18, 47616.92)
(47634.9, 47626.6, 47643.2)
(47597.48, 47601.68, 47593.28)
(47603.37, 47605.62, 47601.12)
(47606.05, 47607.23, 47604.87)
(47619.07, 47625.72, 47612.42)
(47615.45, 47615.32, 47615.58)
(47623.82, 47624.1, 47623.54)
(47585.43, 47593.72, 47577.14)
(47583.0, 47576.43, 47589.57)
(47590.78, 47593.12, 47588.44)
(47591.0, 47598.62, 47583.38)
(47615.9, 47633.57, 47598.23)
(47622.43, 47589.3, 47655.56)
(47483.92, 47475.72, 47492.12)
(47541.5, 47566.88, 47516.12)
(47567.87, 47559.88, 47575.86)
(47528.17, 47515.1, 47541.24)
(47485.03, 47487.32, 47482.74)
(47494.42, 47492.57, 47496.27)
(47506.17, 47509.98, 47502.36)
(47498.67, 47504.28, 47493.06)
(47532.15, 47534.53, 47529.77)
(47493.03, 47486.38, 47499.68)
(47470.13, 47471.8, 47468.46)
(47500.07, 47500.38, 47499.76)
(47547.62, 47549.07, 47546.17)
(47551.18, 47551.35, 47551.01)
(47558.05, 47563.23, 47552.87)
(47562.55, 47561.6, 47563.5)
(47568.65, 47567.1, 47570.2)
(47564.45, 47564.75, 47564.15)
(47539.62, 47527.6, 475

In [14]:
from streaming_indicators import CPR, SuperTrend, EMA
cpr_obj = CPR()
ema_long = EMA(period=6)
ema_short = EMA(period=1)
st = SuperTrend(atr_length=1, factor=2.1)

def apply_indicators(row_as_dict):
    _cpr, _bc, _tc = cpr_obj.update(row_as_dict)
    _ema_long = ema_long.update(row_as_dict["close"])
    _ema_short = ema_short.update(row_as_dict["close"])
    _st, _final_band = st.update(row_as_dict)
    return _cpr, _bc, _tc, _ema_long, _ema_short, _st, _final_band

df[["cpr", "bc", "tc", "ema_60m", "ema_15m", "ST Direction", "ST Value"]] = df.apply(lambda row: apply_indicators(row.to_dict()), axis='columns', result_type='expand')
df

,time,open,high,low,close,cpr,bc,tc,ema_60m,ema_15m,ST Direction,ST Value
114,26-03-2024 09:15:00,46638.75,46782.15,46638.75,46738.05,46719.65,46710.45,46728.85,NaN,46738.05,1.0,46409.31
113,26-03-2024 09:30:00,46740.50,46778.40,46684.10,46713.45,46725.32,46731.25,46719.39,NaN,46713.45,1.0,46533.22
112,26-03-2024 09:45:00,46714.10,46738.15,46667.95,46689.30,46698.47,46703.05,46693.89,NaN,46689.30,1.0,46555.63
111,26-03-2024 10:00:00,46688.25,46716.30,46594.05,46638.15,46649.50,46655.18,46643.82,NaN,46638.15,1.0,46555.63
110,26-03-2024 10:15:00,46634.55,46675.50,46605.50,46661.80,46647.60,46640.50,46654.70,NaN,46661.80,1.0,46555.63
...,...,...,...,...,...,...,...,...,...,...,...,...
4,02-04-2024 11:45:00,47607.05,47627.65,47586.80,47603.70,47606.05,47607.23,47604.87,47601.156102,47603.70,1.0,47547.18
3,02-04-2024 12:00:00,47603.85,47628.95,47582.30,47598.85,47603.37,47605.62,47601.12,47600.497216,47598.85,1.0,47547.18
2,02-04-2024 12:15:00,47597.65,47624.10,47579.25,47589.10,47597.48,47601.68,47593.28,47597.240868,47589.10,1.0,47547.18
1,02-04-2024 12:30:00,47590.10,47663.10,47590.10,47651.50,47634.90,47626.60,47643.20,47612.743477,47651.50,1.0,47547.18


In [15]:
df.to_csv("lite_suite_v1.csv", index=False)